In [1]:
# -*- coding: utf-8 -*-
import sys
import os       
import pandas as pd
import datetime

os.chdir('D:/zxdf/Workspaces/PycharmProjects/flask_ml')
from app import config, Elasticsearch_Util
#from app.lib import nltk_text_analyze_lib as textAnalyzer
#from app.lib import sklearn_classification_lib as skclassification

starttime = datetime.datetime.now()
print ("当前的日期和时间是 %s" % starttime)

In [2]:
def getMoreDataFromES(self, index, doc_type,beginTime, endTime, source_include,size):
    querybody = {
        "query": {
            "match_all": {}
        },
        "_source": {
            "includes": source_include,
            "excludes": []
        }
    }
    # Initialize the scroll
    print "Initialize the scroll"
    page = self.es.search(
        index=index,
        doc_type=doc_type,
        scroll='2m',
        size=size,
        body=querybody)
    sid = page['_scroll_id']
    scroll_size = page['hits']['total']
    
    print "sid: " + str(sid)
    print "scroll size: " + str(scroll_size)
    
    records = []
    for doc in page['hits']['hits']:
            source=doc['_source']
            source["id"]=doc['_id']
            records.append(source)
    
    df = None
   #Start scrolling
    while (scroll_size > 0):
        print "Scrolling..."
        page = self.es.scroll(scroll_id=sid, scroll='2m')
        # Update the scroll ID
        sid = page['_scroll_id']
        print "sid: " + str(sid)
        # Get the number of results that we returned in the last scroll
        scroll_size = len(page['hits']['hits'])
        
        for doc in page['hits']['hits']:
            source=doc['_source']
            source["id"]=doc['_id']
            records.append(source)
            
        print "scroll size: " + str(scroll_size)
        
    if records:
        df = pd.DataFrame(records) 
        # Do something with the obtained page
    return df

In [3]:
index = "1012-knowledge_training_normal-*"
es_type = "knowledge_training_normal"
beginTime = "2018-01-26T00:00:00.000+08:00"
endTime = "2018-01-27T00:00:00.000+08:00"
size = 10000
tag = "isException"
logmsg = "logBody"
source_include = [tag, logmsg]

es_util = Elasticsearch_Util()
train_data_normal = getMoreDataFromES(es_util, index=index, doc_type=es_type,
                           beginTime=beginTime, endTime=endTime,
                           size=size,
                           source_include=source_include)

In [4]:
len(train_data_normal)

692

In [5]:
index = "1012-knowledge_training_error-*"
es_type = "knowledge_training_error"
beginTime = "2018-01-30T00:00:00.000+08:00"
endTime = "2018-02-01T00:00:00.000+08:00"
fromnum = 0
size = 10000
#1085  804:281
tag = "isException"
logmsg = "logBody"
source_include = [tag, logmsg]

es_util = Elasticsearch_Util()
train_data_error = getMoreDataFromES(es_util, index=index, doc_type=es_type,
                           beginTime=beginTime, endTime=endTime,
                           size=size,
                           source_include=source_include)

In [6]:
len(train_data_error)

804

In [7]:
frames=[train_data_normal,train_data_error]
train_data=pd.concat(frames)

In [8]:
len(train_data)

1496

In [9]:
feature_data_array = train_data[logmsg]
class_data_array = train_data[tag]
print len(feature_data_array)
print len(class_data_array)

In [10]:
import nltk
import string
from nltk.corpus import stopwords
#from nltk.stem.porter import *

#训练数据关键词提取列表
#def tokenize(text):
#    f = open(r"E:\zxdf\ml\linkdata-log\dataset\testrecord\keyword_logbody.txt", "a+")
#    tokens = nltk.word_tokenize(text)
#    tokens2=[]
#    for w in tokens:
#        if w[0]=='-':
#            tokens2.append(w[1:])
#        elif w[0]=="'":
#            tokens2.append(w[1:])    
#        else:
#            tokens2.append(w)
#    
#    stopword_tokens = [i for i in tokens2 if i not in string.punctuation]
#    stopword_tokens2 = [w for w in stopword_tokens if w not in stopwords.words('english')]
#    
#    stopwords_custom=["''", "``","||","'/","'/'",u"'0x0","'2","'=","'s",'**','***','***.***','**constraint.unique_wt**','**failed','**method','--',
#                      '-1','-297991290629036654','-614','-6182496564283649260','-6787310117729693199','-8','-999999999',u'-e',
#                      '..','...','.\\xxx\\xxx.txt','/*','/**','/c','/p','0','1','==','b','c','e','x','===',
#                      'v'
#                     ]
#    #,'mapped'
#    stopword_tokens3 = [w for w in stopword_tokens2 if w not in stopwords_custom]    
#            
#    print >> f, "%s" %(stopword_tokens3) 
#    f.close()
#    return stopword_tokens3

In [ ]:
#from sklearn.feature_extraction.text import CountVectorizer
#vectorizer=CountVectorizer(tokenizer=tokenize, stop_words='english', decode_error='ignore')
#csr_mat = vectorizer.fit_transform(feature_data_array)
## 获取词袋模型中的所有词
#wordlist = vectorizer.get_feature_names()  
## tf矩阵 元素a[i][j]表示j词在i类文本中的tf
#countlist = csr_mat.toarray()

In [11]:
# 提取特征
#from sklearn.feature_extraction.text import TfidfTransformer
#tf_transformer=TfidfTransformer()
#feature_datas= tf_transformer.fit_transform(csr_mat)
#
## 获取词袋模型中的所有词
##tfidfwordlist =tf_transformer.get_feature_names()
## tf-idf矩阵 元素a[i][j]表示j词在i类文本中的tf-idf权重
#weighttlist = feature_datas.toarray()
from app.lib import nltk_text_analyze_lib as textAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
tf_transformer = TfidfVectorizer(tokenizer=textAnalyzer.tokenize, stop_words='english', decode_error='ignore')
feature_datas = tf_transformer.fit_transform(feature_data_array)

In [ ]:
#feature_datas.todense()

In [12]:
#验证算法
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

#from sklearn.svm import SVC
#from sklearn.linear_model import LogisticRegression
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#from sklearn.naive_bayes import GaussianNB

def compareAlgorithm(X_train,Y_train,num_folds=10, seed=7, scoring='accuracy'):
    models = {}
    models['KNN'] = KNeighborsClassifier()
    models['CART'] = DecisionTreeClassifier()
    #models['SVM'] = SVC()
    #models['LR'] = LogisticRegression()
    #models['LDA'] = LinearDiscriminantAnalysis()
    #models['NB'] = GaussianNB()
    # 评估算法
    results = []
    for key in models:
        kfold = KFold(n_splits=num_folds, random_state=seed)
        cv_results = cross_val_score(models[key], X_train, Y_train, cv=kfold, scoring=scoring)
        results.append(cv_results)
        print('%s: %f (%f)' %(key, cv_results.mean(), cv_results.std()))
    return models, results
#标准偏差

In [15]:
compareAlgorithm(X_train=feature_datas, Y_train=class_data_array)

({'CART': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, presort=False, random_state=None,
              splitter='best'),
  'KNN': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
             metric_params=None, n_jobs=1, n_neighbors=5, p=2,
             weights='uniform')},
 [array([ 1.        ,  1.        ,  1.        ,  1.        ,  0.61333333,
          0.04666667,  0.03355705,  0.02684564,  0.02013423,  0.05369128]),
  array([ 0.97333333,  0.99333333,  0.96666667,  1.        ,  0.84666667,
          0.42      ,  0.43624161,  0.48993289,  0.42281879,  0.48993289])])

In [16]:
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import joblib

le = LabelEncoder()
train_labelValues = le.fit_transform(class_data_array)

model = DecisionTreeClassifier()
model.fit(feature_datas, train_labelValues)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [ ]:
# teststack6678_804
#app/rules/modelspkl
#D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl
#joblib.dump(tf_transformer, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack6678_804\knowledge_tf_transformer.pkl')
#joblib.dump(le, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack6678_804\knowledge_labelencoder.pkl')
#joblib.dump(model, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack6678_804\knowledge_cart.pkl')


# teststack2527_804
#app/rules/modelspkl
#D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl
#joblib.dump(tf_transformer, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack2527_804\knowledge_tf_transformer.pkl')
#joblib.dump(le, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack2527_804\knowledge_labelencoder.pkl')
#joblib.dump(model, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack2527_804\knowledge_cart.pkl')

# teststack1662_804
#joblib.dump(tf_transformer, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack1662_804\knowledge_tf_transformer.pkl')
#joblib.dump(le, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack1662_804\knowledge_labelencoder.pkl')
#joblib.dump(model, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack1662_804\knowledge_cart.pkl')


# teststack489_804
#joblib.dump(tf_transformer, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack489_804\knowledge_tf_transformer.pkl')
#joblib.dump(le, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack489_804\knowledge_labelencoder.pkl')
#joblib.dump(model, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack489_804\knowledge_cart.pkl')

# teststack290_804
#joblib.dump(tf_transformer, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack290_804\knowledge_tf_transformer.pkl')
#joblib.dump(le, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack290_804\knowledge_labelencoder.pkl')
#joblib.dump(model, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack290_804\knowledge_cart.pkl')

# teststack150_804
#joblib.dump(tf_transformer, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack150_804\knowledge_tf_transformer.pkl')
#joblib.dump(le, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack150_804\knowledge_labelencoder.pkl')
#joblib.dump(model, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\teststack150_804\knowledge_cart.pkl')

In [ ]:
# normal_preprocessing01\teststack22371_804
#joblib.dump(tf_transformer, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\normal_preprocessing01\teststack22371_804\knowledge_tf_transformer.pkl')
#joblib.dump(le, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\normal_preprocessing01\teststack22371_804\knowledge_labelencoder.pkl')
#joblib.dump(model, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\normal_preprocessing01\teststack22371_804\knowledge_cart.pkl')


# normal_preprocessing01\teststack773_804
#joblib.dump(tf_transformer, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\normal_preprocessing01\teststack773_804\knowledge_tf_transformer.pkl')
#joblib.dump(le, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\normal_preprocessing01\teststack773_804\knowledge_labelencoder.pkl')
#joblib.dump(model, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\normal_preprocessing01\teststack773_804\knowledge_cart.pkl')


In [18]:
# normal_preprocessing02\teststack1618_804
#joblib.dump(tf_transformer, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\normal_preprocessing02\teststack1618_804\knowledge_tf_transformer.pkl')
#joblib.dump(le, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\normal_preprocessing02\teststack1618_804\knowledge_labelencoder.pkl')
#joblib.dump(model, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\normal_preprocessing02\teststack1618_804\knowledge_cart.pkl')

# normal_preprocessing02\teststack1043_804
#joblib.dump(tf_transformer, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\normal_preprocessing02\teststack1043_804\knowledge_tf_transformer.pkl')
#joblib.dump(le, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\normal_preprocessing02\teststack1043_804\knowledge_labelencoder.pkl')
#joblib.dump(model, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\normal_preprocessing02\teststack1043_804\knowledge_cart.pkl')

# normal_preprocessing02\teststack692_804
joblib.dump(tf_transformer, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\normal_preprocessing02\teststack692_804\knowledge_tf_transformer.pkl')
joblib.dump(le, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\normal_preprocessing02\teststack692_804\knowledge_labelencoder.pkl')
joblib.dump(model, r'D:\zxdf\Workspaces\PycharmProjects\flask_ml\app\rules\modelspkl\normal_preprocessing02\teststack692_804\knowledge_cart.pkl')


['D:\\zxdf\\Workspaces\\PycharmProjects\\flask_ml\\app\\rules\\modelspkl\\normal_preprocessing02\\teststack692_804\\knowledge_cart.pkl']

In [ ]:
#model = DecisionTreeClassifier()
#model.fit(feature_datas, class_data_array)

In [ ]:
#testline='-Mapped java.lang.StringIndexOutOfBoundsException: String index out of range: 0 '
#
#test_datas = []
#featureValue = testline.lower().strip()
#test_datas.append(featureValue)
##test_feature_datas = tf_transformer.transform(test_datas)
#test_vect_datas=vectorizer.transform(test_datas)
#test_feature_datas= tf_transformer.transform(test_vect_datas)
#model.predict(test_feature_datas)